In [15]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# UR5 robot dimensions (in meters)
d1 = 0.089159
a2 = -0.42500
a3 = -0.39225
d4 = 0.10915
d5 = 0.09465
d6 = 0.0823

# Denavit-Hartenberg parameters for UR5
# theta, d, a, alpha
dh_parameters = [
    {'theta': 0, 'd': d1, 'a': 0, 'alpha': np.pi/2},
    {'theta': 0, 'd': 0, 'a': a2, 'alpha': 0},
    {'theta': 0, 'd': 0, 'a': a3, 'alpha': 0},
    {'theta': 0, 'd': d4, 'a': 0, 'alpha': np.pi/2},
    {'theta': 0, 'd': 0, 'a': 0, 'alpha': -np.pi/2},
    {'theta': 0, 'd': d5, 'a': 0, 'alpha': 0}
]

def transformation_matrix(theta, d, a, alpha):
    """
    Create the transformation matrix using the Denavit-Hartenberg parameters
    """
    return np.array([
        [np.cos(theta), -np.sin(theta)*np.cos(alpha), np.sin(theta)*np.sin(alpha), a*np.cos(theta)],
        [np.sin(theta), np.cos(theta)*np.cos(alpha), -np.cos(theta)*np.sin(alpha), a*np.sin(theta)],
        [0, np.sin(alpha), np.cos(alpha), d],
        [0, 0, 0, 1]
    ])

def forward_kinematics(joint_angles):
    """
    Calculate the forward kinematics for the given joint angles of the UR5 robot
    """
    T = np.eye(4)
    for i, angle in enumerate(joint_angles):
        dh = dh_parameters[i]
        T = np.dot(T, transformation_matrix(angle + dh['theta'], dh['d'], dh['a'], dh['alpha']))
    return T

def calculate_joint_positions(joint_angles):
    """
    Calculate the positions of each joint in the robot arm
    """
    positions = [(0, 0, 0)]  # Base position
    T = np.eye(4)
    for i, angle in enumerate(joint_angles):
        dh = dh_parameters[i]
        T = np.dot(T, transformation_matrix(angle + dh['theta'], dh['d'], dh['a'], dh['alpha']))
        x, y, z = T[:3, 3]
        positions.append((x, y, z))
    return positions

def plot_arm(joint_positions):
    """
    Plot the robotic arm in 3D space
    """
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    xs, ys, zs = zip(*joint_positions)
    ax.scatter(xs, ys, zs, color='red')  # Joints
    ax.plot(xs, ys, zs, color='blue')    # Links
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('UR5 Robotic Arm Configuration')
    plt.show()

# Example joint angles (in radians)
joint_angles = [0, -np.pi/4, np.pi/3, -np.pi/2, np.pi/4, np.pi/6]

# Calculate joint positions
joint_positions = calculate_joint_positions(joint_angles_series[0])

# Plot the robotic arm
plot_arm(joint_positions)



In [13]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation

%matplotlib qt
def interpolate_joint_angles(start_angles, end_angles, steps):
    """
    Linearly interpolate between two sets of joint angles
    """
    return [np.linspace(start, end, steps) for start, end in zip(start_angles, end_angles)]

def generate_joint_angle_series(start_angles, end_angles, steps):
    """
    Generate a series of joint angles for the animation
    """
    interpolated_angles = interpolate_joint_angles(start_angles, end_angles, steps)
    return np.array(interpolated_angles).T


# [Include the previous functions here: transformation_matrix, forward_kinematics, calculate_joint_positions]

def update_animation(frame, joint_angles_series, line, scatter):
    """
    Update function for the animation
    """
    joint_angles = joint_angles_series[frame]
    joint_positions = calculate_joint_positions(joint_angles)
    xs, ys, zs = zip(*joint_positions)
    line.set_data(xs, ys)
    line.set_3d_properties(zs)
    scatter._offsets3d = (xs, ys, zs)
    return line, scatter

def animate_ur5(joint_angles_series):
    """
    Animate the UR5 robotic arm with a series of joint angles
    """
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('UR5 Robotic Arm Animation')

    # Initial joint positions
    joint_positions = calculate_joint_positions(joint_angles_series[0])
    xs, ys, zs = zip(*joint_positions)
    
    line, = ax.plot(xs, ys, zs, color='blue')
    scatter = ax.scatter(xs, ys, zs, color='red')

    # Creating the animation
    anim = FuncAnimation(fig, update_animation, frames=len(joint_angles_series), 
                         fargs=(joint_angles_series, line, scatter), interval=100)

    plt.show()
    return anim

# Example series of joint angles (replace with your own series)
joint_angles_series = [
    [0, -np.pi/4, np.pi/3, -np.pi/2, np.pi/4, np.pi/6],
    [np.pi/6, -np.pi/3, np.pi/4, -np.pi/2, np.pi/6, 0],
    # Add more joint angle sets for a longer animation
]

# Animate the UR5 robotic arm
animate_ur5(joint_angles_series)


In [ ]:
anim.save('ur5_animation.mp4', writer='ffmpeg')

In [7]:
file_path = 'joint.npy'
data = np.load(file_path, allow_pickle=True).item()
# Assuming you want to use the joint data of the first serial number in the file
serial_number = list(data.keys())[0]
joint_data = data[serial_number]

# Extract joint angles in sorted order of timestamps
timestamps = sorted(joint_data.keys())
joint_angles_series = np.array([joint_data[t] for t in timestamps])
joint_angles_series.shape

(199, 6)

In [14]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation

# Load the joint angles from the provided file
file_path = 'joint.npy'
data = np.load(file_path, allow_pickle=True).item()

# Extracting joint angles for the first serial number
serial_number = list(data.keys())[0]
joint_data = data[serial_number]

# Sorting and extracting joint angles in the order of timestamps
timestamps = sorted(joint_data.keys())
joint_angles_series = np.array([joint_data[t] for t in timestamps])

# Define UR5 robot dimensions (in meters) and DH parameters...
# [Include the DH parameter definitions, transformation_matrix, and calculate_joint_positions functions here]

def update_animation(frame, joint_angles_series, line, scatter):
    """
    Update function for the animation
    """
    joint_angles = joint_angles_series[frame]
    joint_positions = calculate_joint_positions(joint_angles)
    xs, ys, zs = zip(*joint_positions)
    line.set_data(xs, ys)
    line.set_3d_properties(zs)
    scatter._offsets3d = (xs, ys, zs)
    return line, scatter

def animate_ur5(joint_angles_series):
    """
    Animate the UR5 robotic arm with a series of joint angles
    """
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('UR5 Robotic Arm Animation')

    # Initial joint positions
    joint_positions = calculate_joint_positions(joint_angles_series[0])
    xs, ys, zs = zip(*joint_positions)
    
    line, = ax.plot(xs, ys, zs, color='blue')
    scatter = ax.scatter(xs, ys, zs, color='red')

    # Creating the animation
    anim = FuncAnimation(fig, update_animation, frames=len(joint_angles_series), 
                         fargs=(joint_angles_series, line, scatter), interval=100)

    plt.show()
    return anim

# Animate the UR5 robotic arm using the loaded joint angles
animate_ur5(joint_angles_series)
